In [1]:
from himalaya.ridge import RidgeCV
import numpy as np
from pathlib import Path
from AOTaccess.stimulus_info_access import StimuliInfoAccess
from AOTaccess.glmsingle_access import GLMSingleAccess

from himalaya.backend import set_backend

# Set backend with a fallback for numpy operations
backend = set_backend("torch_cuda", on_error="warn")

from AOTanalysis.bandedRR.utils import (
    reshape_from_flatten_masked_to_wholebrain,
)
from AOTanalysis.voxelsemantic.corpus_construct import construct_AOT_corpus
import joblib
from pprint import pprint

import matplotlib.pyplot as plt
import random
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from nibabel import Nifti1Image

import cortex
import torch

from AOTanalysis.voxelmotion.filter_info import FilterInfo

/tank/zhangs/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/tank/zhangs/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
sub = 3
model_path = f"/tank/shared/2024/visual/AOT/temp/bandedRR_split_single_feature_withSTD_session_testinside_old_Ycentered/model_sub{sub}_feature_motion32_trainses_1_Xcentered_True_Ycentered_True_Xstd_True_testinside.joblib"
model = joblib.load(model_path)

In [3]:
primal_coef = model[-1].get_primal_coef()
# Keep as a GPU tensor if possible
if hasattr(primal_coef, 'is_cuda') and primal_coef.is_cuda:
    # Already on GPU, great!
    pass
else:
    # Need to convert to numpy then to a GPU tensor
    primal_coef = backend.to_numpy(primal_coef)

print("(n_features, n_voxels) =", primal_coef.shape)



(n_features, n_voxels) = (11845, 352914)


In [4]:
feature_number = primal_coef.shape[0]
voxel_number = primal_coef.shape[1]
filter_info = FilterInfo()
print("feature_number, voxel_number) =", feature_number, voxel_number)

# Convert primal_coef to PyTorch tensor and move to GPU
primal_coef_tensor = torch.tensor(primal_coef, device='cuda')

# Pre-compute polar angles for all features and convert to tensor
polar_angles = np.array([filter_info.index_to_polar_angle(j) for j in range(feature_number)])
polar_angles_tensor = torch.tensor(polar_angles, device='cuda')

# Calculate absolute sum of coefficients for each voxel
abs_sums = torch.sum(torch.abs(primal_coef_tensor), dim=0)

# Handle zero sums to avoid division by zero
valid_sums_mask = abs_sums > 0

# Create normalized coefficients (vectorized operation)
normalized_coefs = torch.zeros_like(primal_coef_tensor)
normalized_coefs[:, valid_sums_mask] = primal_coef_tensor[:, valid_sums_mask] / abs_sums[valid_sums_mask]

# Calculate weighted sum of polar angles (vectorized matrix multiplication)
# Shape: [n_features, n_voxels] * [n_features, 1] = [n_voxels]
average_polar_angles = torch.matmul(normalized_coefs.T, polar_angles_tensor.unsqueeze(1)).squeeze(1)

# Convert back to numpy for further processing
average_polar_angles = average_polar_angles.cpu().numpy()

# Reshape to whole brain volume
average_polar_angles_volume = reshape_from_flatten_masked_to_wholebrain(average_polar_angles, sub)
print("average_polar_angles_volume.shape =", average_polar_angles_volume.shape)

# Transpose the volume to match the expected shape (106, 95, 84)
average_polar_angles_volume = np.transpose(average_polar_angles_volume, (2, 1, 0))
print("Transposed polar angles volume shape:", average_polar_angles_volume.shape)

glminfo = GLMSingleAccess()
affine = glminfo.read_affine(sub)
header = glminfo.read_header(sub)

save_path = Path(f"/tank/shared/2024/visual/AOT/temp/motion_energy_analysis/motion2polar_average")
save_path.mkdir(parents=True, exist_ok=True)
nifti_img = Nifti1Image(average_polar_angles_volume, affine=affine, header=header)
nifti_img.to_filename(save_path / f"average_polar_angles_sub{sub}_optimize_test.nii.gz")



feature_number, voxel_number) = 11845 352914


OutOfMemoryError: CUDA out of memory. Tried to allocate 15.57 GiB. GPU 0 has a total capacity of 14.57 GiB of which 14.21 GiB is free. Process 3778981 has 120.00 MiB memory in use. Process 3799465 has 120.00 MiB memory in use. Including non-PyTorch memory, this process has 120.00 MiB memory in use. Of the allocated memory 1.35 MiB is allocated by PyTorch, and 18.65 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Note: The polar_angles_volume has been transposed to match the expected shape for cortex.Volume
sub = "sub-003"
volume = cortex.Volume(average_polar_angles_volume, sub, "AOT1pt7mm", recache=True)
mapper = cortex.get_mapper(sub, "AOT1pt7mm", type="nearest", recache=True)
native_surface_map = mapper(volume)

# Set colormap
selected_cmap = "hsv"  # Change this to your desired colormap
native_surface_map.cmap = selected_cmap

cortex.quickshow(
    native_surface_map,
    with_curvature=True,
    with_colorbar=True,
    with_labels=False,
    with_sulci=True,
    with_legend=False,
    cmap=selected_cmap,
    colorbar_label="Polar Angle",
)

In [ ]:
# Benchmarking cell to compare original vs optimized approach
import time
from functools import wraps

def timing_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Function {func.__name__} took {end_time - start_time:.4f} seconds to execute")
        return result
    return wrapper

@timing_decorator
def original_implementation(primal_coef, filter_info):
    feature_number = primal_coef.shape[0]
    voxel_number = primal_coef.shape[1]
    average_polar_angles = np.zeros((voxel_number))
    
    for i in range(voxel_number):
        voxel_coefs = primal_coef[:, i]
        abs_sum = torch.sum(torch.abs(torch.tensor(voxel_coefs))).item()
        voxel_coefs_normalized = voxel_coefs / abs_sum if abs_sum > 0 else np.zeros_like(voxel_coefs)
        
        average_polar_angle = 0
        for j in range(feature_number):
            polar_angle = filter_info.index_to_polar_angle(j)
            average_polar_angle += voxel_coefs_normalized[j] * polar_angle
        
        average_polar_angles[i] = average_polar_angle
    
    return average_polar_angles

@timing_decorator
def optimized_implementation(primal_coef, filter_info):
    feature_number = primal_coef.shape[0]
    voxel_number = primal_coef.shape[1]
    
    # Convert to PyTorch tensor and move to GPU
    if not isinstance(primal_coef, torch.Tensor):
        primal_coef_tensor = torch.tensor(primal_coef, device='cuda')
    else:
        primal_coef_tensor = primal_coef.to('cuda')
    
    # Pre-compute polar angles for all features
    polar_angles = np.array([filter_info.index_to_polar_angle(j) for j in range(feature_number)])
    polar_angles_tensor = torch.tensor(polar_angles, device='cuda').unsqueeze(1)
    
    # Calculate absolute sum of coefficients
    abs_sums = torch.sum(torch.abs(primal_coef_tensor), dim=0)
    
    # Create normalized coefficients with zero handling
    normalized_coefs = torch.zeros_like(primal_coef_tensor)
    valid_mask = abs_sums > 0
    normalized_coefs[:, valid_mask] = primal_coef_tensor[:, valid_mask] / abs_sums[valid_mask]
    
    # Calculate weighted sum of polar angles (vectorized)
    average_polar_angles = torch.matmul(normalized_coefs.T, polar_angles_tensor).squeeze(1)
    
    return average_polar_angles.cpu().numpy()

# Run both implementations with a small subset to verify correctness
small_coef = primal_coef[:, :1000] if isinstance(primal_coef, np.ndarray) else backend.to_numpy(primal_coef)[:, :1000]

print("Testing with a subset of", small_coef.shape[1], "voxels")
#original_result = original_implementation(small_coef, filter_info)
optimized_result = optimized_implementation(small_coef, filter_info)

# # Check if results match
# print("Results match:", np.allclose(original_result, optimized_result, rtol=1e-5, atol=1e-5))
# print("Mean absolute difference:", np.mean(np.abs(original_result - optimized_result)))